Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [14]:
import random
from collections import namedtuple
from tqdm import tqdm
from icecream import ic

In [15]:
PROBLEM_SIZE = 100

In [16]:
Individual = namedtuple('Individual', ['genotype', 'fitness'])

In [48]:
def fitness_function(sol: list[bool]) -> int:
    # return max(sum(sol), PROBLEM_SIZE - sum(sol))
    return sum(sol)


def single_bit_mutation(sol: Individual):
    new_genotype = sol.genotype[:]
    idx = random.randint(0, PROBLEM_SIZE - 1)
    new_genotype[idx] = not new_genotype[idx]
    return Individual(new_genotype, None)


def one_cut_xover(p1: Individual, p2: Individual) -> Individual:
    idx = random.randint(0, PROBLEM_SIZE - 1)
    new_genotype = p1.genotype[:idx] + p2.genotype[idx:]
    return Individual(new_genotype, None)

In [77]:
def fitness_proportional(population: list[Individual]) -> Individual:
    tot_fitness = sum(i.fitness for i in population)
    weights = [i.fitness / tot_fitness for i in population]
    s = random.choices(population, weights=weights)
    return s[0]


def tournament_selection(population: list[Individual], tau: int = 5) -> Individual:
    pool = random.choices(population, k=tau)
    return max(pool, key=lambda i: i.fitness)


select_parent = tournament_selection

In [78]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20

In [79]:
MUTATION_RATE = 0.5

In [80]:
for stats in range(10):
    current_population = list()
    for i in range(POPULATION_SIZE):
        genotype = [random.random() < 0.5 for _ in range(PROBLEM_SIZE)]
        fitness = fitness_function(genotype)
        new_individual = Individual(genotype, fitness)
        current_population.append(new_individual)

    best_fitness = -1
    generation = 0
    evaluations = 0
    while best_fitness < PROBLEM_SIZE:
        generation += 1
        offspring = list()
        for o in range(OFFSPRING_SIZE):
            if random.random() < MUTATION_RATE:
                p = select_parent(current_population)
                o = single_bit_mutation(p)
            else:
                p1 = select_parent(current_population)
                p2 = select_parent(current_population)
                o = one_cut_xover(p1, p2)
            offspring.append(o)
        evaluations += len(offspring)
        offspring = [Individual(o.genotype, fitness_function(o.genotype)) for o in offspring]

        # steady state
        current_population.extend(offspring)

        # survival selection
        current_population = sorted(current_population, key=lambda i: i.fitness, reverse=True)
        if current_population[0].fitness > best_fitness:
            best_fitness = current_population[0].fitness

        current_population = current_population[:POPULATION_SIZE]
    ic(generation, evaluations, best_fitness)

ic| generation: 84, evaluations: 1680, best_fitness: 100


ic| generation: 88, evaluations: 1760, best_fitness: 100
ic| generation: 73, evaluations: 1460, best_fitness: 100
ic| generation: 94, evaluations: 1880, best_fitness: 100
ic| generation: 93, evaluations: 1860, best_fitness: 100
ic| generation: 106, evaluations: 2120, best_fitness: 100
ic| generation: 87, evaluations: 1740, best_fitness: 100
ic| generation: 80, evaluations: 1600, best_fitness: 100
ic| generation: 79, evaluations: 1580, best_fitness: 100
ic| generation: 129, evaluations: 2580, best_fitness: 100


In [9]:
len(offspring)

4